# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
collection_names = db.list_collection_names()
print(collection_names)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
#preview dataset
document = establishments.find_one()
pprint(document)

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
results = establishments.find(query)

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
rows = df.shape[0]
print("The number of rows in the DataFrame is ", rows)

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": ".*London.*"}, "RatingValue": {"$gte": 4}}
results = establishments.find(query)

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
rows = df.shape[0]
print("The number of rows in the DataFrame is ", rows)

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

# Calculate the latitude and longitude ranges
latitude_range = [latitude - degree_search, latitude + degree_search]
longitude_range = [longitude - degree_search, longitude + degree_search]


#build query
query = {
    "$and": [
        {"geocode.latitude": {"$gte": latitude_range[0], "$lte": latitude_range[1]}},
        {"geocode.longitude": {"$gte": longitude_range[0], "$lte": longitude_range[1]}},
        {"RatingValue": 5}
    ]
}

sort = [("scores.Hygiene", 1)]

results = establishments.find(query).sort(sort)  
documents = list(results)
pprint(documents)

In [ ]:
# Use count_documents to display the number of documents in the result
print("Number of documents in result:", establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(documents[0])

In [ ]:
# Convert result to Pandas DataFrame
penang_df = pd.DataFrame(documents)
penang_df

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

result = establishments.aggregate(pipeline)
result_list = list(result)

# Print the number of documents in the result
print("Number of documents in the result:", len(result_list))

# Print the first 10 results
for doc in result_list[:10]:
    print(doc)

In [ ]:
# Convert the result to a Pandas DataFrame
pipeline_df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
num_rows = pipeline_df.shape[0]
print("Number of rows:", num_rows)

# Display the first 10 rows of the DataFrame
pipeline_df.head(10)